In [98]:
import numpy as np
import pandas as pd
import urllib

import cmocean
import folium

def cmocean_to_leaflet(cmap, pl_entries):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []
    
    for k in range(pl_entries):
        C = list(map(np.uint8, np.array(cmap(k*h)[:3])*255))
        pl_colorscale.append([k*h, "#{:02x}{:02x}{:02x}".format(C[0], C[1], C[2])])
        
    return pl_colorscale

thermal = cmocean_to_leaflet(cmocean.cm.thermal, 1000)

def rgb2hex(r,g,b):
    hex = "#{:02x}{:02x}{:02x}".format(r,g,b)
    return hex

def color_normalize(cmin,cmax,value):
    return (value-cmin)/(cmax-cmin)

In [99]:
"""--------------------- Get Data  -----------------------------------------"""

### using code from https://ioos.github.io/notebooks_demos/notebooks/2017-03-21-ERDDAP_IOOS_Sensor_Map/
import requests
try:
    from urllib.parse import urlencode
except ImportError:
    from urllib import urlencode


def encode_erddap(urlbase, fname, columns, params):
    """
    urlbase: the base string for the endpoint
             (e.g.: https://erddap.axiomdatascience.com/erddap/tabledap).
    fname: the data source (e.g.: `sensor_service`) and the response (e.g.: `.csvp` for CSV).
    columns: the columns of the return table.
    params: the parameters for the query.

    Returns a valid ERDDAP endpoint.
    """
    urlbase = urlbase.rstrip('/')
    if not urlbase.lower().startswith(('http:', 'https:')):
        msg = 'Expected valid URL but got {}'.format
        raise ValueError(msg(urlbase))

    columns = ','.join(columns)
    params = urlencode(params)
    endpoint = '{urlbase}/{fname}?{columns}&{params}'.format

    url = endpoint(urlbase=urlbase, fname=fname,
                   columns=columns, params=params)
    r = requests.get(url)
    r.raise_for_status()
    return url

try:
    from urllib.parse import unquote
except ImportError:
    from urllib2 import unquote

# return profile for float id
def get_profile(alamo_id,starttime,endtime):
    
    urlbase = 'http://ferret.pmel.noaa.gov/alamo/erddap/tabledap'

    fname = 'arctic_heat_alamo_profiles_'+str(alamo_id)+'.csvp'

    columns = ('profileid',
               'FLOAT_SERIAL_NO',
               'CYCLE_NUMBER',
               'REFERENCE_DATE_TIME',
               'JULD',
               'time',
               'latitude',
               'longitude',
               'PRES',
               'TEMP',
               'PSAL')
    params = {
        # Inequalities do not exist in HTTP parameters,
        # so we need to hardcode the `>` in the time key to get a '>='.
        # Note that a '>' or '<' cannot be encoded with `urlencode`, only `>=` and `<=`.
        'time>': starttime+'T00:00:00Z',
        'time<': endtime+'T00:00:00Z',
        'PRES>': 0.5,
        'PRES<': 35,
    }

    url = encode_erddap(urlbase, fname, columns, params)

    df = pd.read_csv(url, index_col=0, parse_dates=['time (UTC)'])
    
    return df

## Data is ingested from ERDDAP Server to PANDAS

df = get_profile(9119,'2017-08-01','2017-12-31')
dfgroup = df.groupby('CYCLE_NUMBER')
dfgbc = dfgroup.mean()
dfgbc['time (UTC)'] = dfgroup.first()['time (UTC)']
dfgbc['doy'] = [x.to_pydatetime().timetuple().tm_yday for x in dfgroup.first()['time (UTC)']]
dfgbc['fracday'] = [((x.to_pydatetime().timetuple().tm_hour*60.) + x.to_pydatetime().timetuple().tm_min)/1440. for x in dfgroup.first()['time (UTC)']]
dfgbc['doyfrac'] = dfgbc['doy'] + dfgbc['fracday']
dfgbc['index_name'] = dfgbc.index
dfgbc = dfgbc.round(4)
dfgbc.drop(['REFERENCE_DATE_TIME','FLOAT_SERIAL_NO'], axis=1, inplace=True)

df2 = get_profile(9076,'2016-08-01','2017-12-31')
df2group = df2.groupby('CYCLE_NUMBER')
df2gbc = df2group.mean()
df2gbc['time (UTC)'] = df2group.first()['time (UTC)']
df2gbc['doy'] = [x.to_pydatetime().timetuple().tm_yday for x in df2group.first()['time (UTC)']]
df2gbc['fracday'] = [((x.to_pydatetime().timetuple().tm_hour*60.) + x.to_pydatetime().timetuple().tm_min)/1440. for x in df2group.first()['time (UTC)']]
df2gbc['doyfrac'] = df2gbc['doy'] + df2gbc['fracday']
df2gbc['index_name'] = df2gbc.index
df2gbc = df2gbc.round(4)
df2gbc.drop(['REFERENCE_DATE_TIME','FLOAT_SERIAL_NO'], axis=1, inplace=True)

df3 = get_profile(9085,'2016-08-01','2016-12-31')
df3group = df3.groupby('CYCLE_NUMBER')
df3gbc = df3group.mean()
df3gbc['time (UTC)'] = df3group.first()['time (UTC)']
df3gbc['doy'] = [x.to_pydatetime().timetuple().tm_yday for x in df3group.first()['time (UTC)']]
df3gbc['fracday'] = [((x.to_pydatetime().timetuple().tm_hour*60.) + x.to_pydatetime().timetuple().tm_min)/1440. for x in df3group.first()['time (UTC)']]
df3gbc['doyfrac'] = df3gbc['doy'] + df3gbc['fracday']
df3gbc['index_name'] = df3gbc.index
df3gbc = df3gbc.round(4)
df3gbc.drop(['REFERENCE_DATE_TIME','FLOAT_SERIAL_NO'], axis=1, inplace=True)


In [100]:
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return idx

tol = .00001



In [101]:
f1 = folium.map.FeatureGroup()

lats = dfgbc['latitude (degrees_north)'].values
lngs = dfgbc['longitude (degrees_east)'].values
colors = dfgbc['TEMP (degree_Celsius)'].values
for lat, lng, color in zip(lats, lngs, colors):
    nval = find_nearest(np.array([x[0] for x in thermal]),color_normalize(-2.,10.,color))
    f1.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=1,
            color=thermal[nval][1],
        )
        )

In [102]:
f2 = folium.map.FeatureGroup()

lats = df2gbc['latitude (degrees_north)'].values
lngs = df2gbc['longitude (degrees_east)'].values
colors = df2gbc['TEMP (degree_Celsius)'].values
for lat, lng, color in zip(lats, lngs, colors):
    nval = find_nearest(np.array([x[0] for x in thermal]),color_normalize(-2.,10.,color))
    f2.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=1,
            color=thermal[nval][1],
        )
        )

In [103]:
f3 = folium.map.FeatureGroup()

lats = df3gbc['latitude (degrees_north)'].values
lngs = df3gbc['longitude (degrees_east)'].values
colors = df3gbc['TEMP (degree_Celsius)'].values
for lat, lng, color in zip(lats, lngs, colors):
    nval = find_nearest(np.array([x[0] for x in thermal]),color_normalize(-2.,10.,color))
    f3.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=1,
            color=thermal[nval][1],
        )
        )

In [104]:
### resample to fixed time grid and fill interpolated temperatures with large value

dfgbc_t = dfgbc.set_index('time (UTC)')
dfgbc_t = dfgbc_t.resample('12H').mean()
dfgbc_t['TEMP (degree_Celsius)'].replace(np.nan,1000,inplace=True)
dfgbc_t=dfgbc_t.interpolate()

f4 = folium.map.FeatureGroup()

lats = dfgbc_t['latitude (degrees_north)'].values
lngs = dfgbc_t['longitude (degrees_east)'].values
colors = dfgbc_t['TEMP (degree_Celsius)'].values
for lat, lng, color in zip(lats, lngs, colors):
    nval = find_nearest(np.array([x[0] for x in thermal]),color_normalize(-2.,10.,color))
    f4.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=1,
            color=thermal[nval][1],
        )
        )
    
f4s = folium.map.FeatureGroup()

lats = dfgbc_t['latitude (degrees_north)'].values[0]
lngs = dfgbc_t['longitude (degrees_east)'].values[0]

f4s.add_child(
    folium.features.CircleMarker(
        [lats, lngs],
        radius=4,
        color='#41897F'
        )
        )

In [105]:
### resample to fixed time grid and fill interpolated temperatures with large value

df2gbc_t = df2gbc.set_index('time (UTC)')
df2gbc_t = df2gbc_t.resample('12H').mean()
df2gbc_t['TEMP (degree_Celsius)'].replace(np.nan,1000,inplace=True)
df2gbc_t=df2gbc_t.interpolate()

f5 = folium.map.FeatureGroup()

lats = df2gbc_t['latitude (degrees_north)'].values
lngs = df2gbc_t['longitude (degrees_east)'].values
colors = df2gbc_t['TEMP (degree_Celsius)'].values
for lat, lng, color in zip(lats, lngs, colors):
    nval = find_nearest(np.array([x[0] for x in thermal]),color_normalize(-2.,10.,color))
    f5.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=1,
            color=thermal[nval][1],
        )
        )

f5s = folium.map.FeatureGroup()

lats = df2gbc_t['latitude (degrees_north)'].values[0]
lngs = df2gbc_t['longitude (degrees_east)'].values[0]

f5s.add_child(
    folium.features.CircleMarker(
        [lats, lngs],
        radius=4,
        color='#18389A',
        popup="2016 9085"
        )
        )

In [106]:
### resample to fixed time grid and fill interpolated temperatures with large value

df3gbc_t = df3gbc.set_index('time (UTC)')
df3gbc_t = df3gbc_t.resample('12H').mean()
df3gbc_t['TEMP (degree_Celsius)'].replace(np.nan,1000,inplace=True)
df3gbc_t=df3gbc_t.interpolate()

f6 = folium.map.FeatureGroup()

lats = df3gbc_t['latitude (degrees_north)'].values
lngs = df3gbc_t['longitude (degrees_east)'].values
colors = df3gbc_t['TEMP (degree_Celsius)'].values
for lat, lng, color in zip(lats, lngs, colors):
    nval = find_nearest(np.array([x[0] for x in thermal]),color_normalize(-2.,10.,color))
    f6.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=1,
            color=thermal[nval][1],
        )
        )
    
f6s = folium.map.FeatureGroup()

lats = df3gbc_t['latitude (degrees_north)'].values[0]
lngs = df3gbc_t['longitude (degrees_east)'].values[0]

f6s.add_child(
    folium.features.CircleMarker(
        [lats, lngs],
        radius=4,
        color='#ED7F33',
        popup="2017 9085"
        )
        )

In [107]:
m = folium.Map(location=[71,-160],
              tiles='https://api.mapbox.com/styles/v1/sbell/cj48p13bt265k2qvucuielhm7/tiles/256/{z}/{x}/{y}?access_token=pk.eyJ1Ijoic2JlbGwiLCJhIjoiY2lqbGlpaHBkMDAyanV5bHhqMTdjYTd5aiJ9.04PwNcY3Piny-YtIg5cIJA',
              attr='North Star - MapBox',
              zoom_start=6)
#m.add_child(f1)
#m.add_child(f2)
#m.add_child(f3)
m.add_child(f4)
m.add_child(f5)
m.add_child(f6)
m.add_child(f4s)
m.add_child(f5s)
m.add_child(f6s)


In [108]:
df3gbc_t

,JULD (UTC),latitude (degrees_north),longitude (degrees_east),PRES (decibar),TEMP (degree_Celsius),PSAL (PSU),doy,fracday,doyfrac,index_name
time (UTC),,,,,,,,,,
2016-09-16 12:00:00,NaN,70.441800,-166.833750,17.974675,4.562700,NaN,260.000000,0.909700,260.909700,6.500000
2016-09-17 00:00:00,NaN,70.442733,-166.813100,17.976800,5.089633,NaN,261.000000,0.205317,261.205317,11.500000
2016-09-17 12:00:00,NaN,70.489600,-166.717300,17.999200,4.830300,NaN,261.000000,0.895100,261.895100,15.000000
2016-09-18 00:00:00,NaN,70.516950,-166.615650,17.951350,4.663250,NaN,262.000000,0.337150,262.337150,16.500000
2016-09-18 12:00:00,NaN,70.566550,-166.527100,17.892600,4.655800,NaN,262.000000,0.817700,262.817700,18.500000
2016-09-19 00:00:00,NaN,70.618850,-166.507750,16.306100,4.589300,NaN,263.000000,0.315250,263.315250,20.500000
2016-09-19 12:00:00,NaN,70.659000,-166.530700,17.815050,4.438700,NaN,263.000000,0.811100,263.811100,22.500000
2016-09-20 00:00:00,NaN,70.680350,-166.570100,17.628550,4.621850,NaN,264.000000,0.311800,264.311800,24.500000
2016-09-20 12:00:00,NaN,70.701400,-166.617650,17.706200,4.577450,NaN,264.000000,0.814600,264.814600,26.500000
